In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tkinter as tk
from tkinter import messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


In [ ]:


final_dataset = pd.read_csv("G:\\anaconda\\data collection\\data set\\final dataset\\Stock_Data_for_Prediction.csv")
final_dataset['Date'] = pd.to_datetime(final_dataset['Date'], errors='coerce', format='%Y-%m-%d')
final_dataset = final_dataset.dropna(subset=['Date'])

# Function to predict stock price based on amount and company name
def predict_stock_price(company_name, amount):

    company_data = final_dataset[final_dataset['Company'] == company_name]
    
    if company_data.empty:
        messagebox.showerror("Error", "Company not found in the dataset")
        return None, None, None, None
    
    # Extract features (date, and close price)
    company_data['Date'] = pd.to_datetime(company_data['Date'])
    company_data['Date'] = company_data['Date'].map(pd.Timestamp.toordinal)
    X = company_data[['Date']]
    y = company_data['Close']

    # Train a Linear Regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict the stock price for the given amount (date)
    last_date = company_data['Date'].max()  # Last available date
    predicted_price = model.predict([[last_date + amount]])  # Predict based on the input amount (offset from last date)

    # Calculate predictions for MSE and MAE calculation
    predictions = model.predict(X)
    mse = mean_squared_error(y, predictions)
    mae = mean_absolute_error(y, predictions)

    return predicted_price[0], amount * company_data['Close'].iloc[-1], mse, mae

# Tkinter window
def plot_stock_chart(company_name, amount_predicted, predicted_price, mse, mae, company_data, canvas_frame):
    
    if len(company_data) > 100:
        company_data = company_data.tail(100)

    
    fig, ax = plt.subplots(figsize=(10, 6))  # Set a reasonable figure size
    ax.plot(company_data['Date'], company_data['Close'], label='Stock Price', color='blue')

    ax.axvline(x=company_data['Date'].max(), color='red', linestyle='--', label='Prediction')

    ax.text(company_data['Date'].max(), predicted_price, f'Predicted Price: ₹{predicted_price:.2f}', color='red')

    ax.set_xlabel('Date')
    ax.set_ylabel('Stock Price (₹)')
    ax.set_title(f'Stock Price Prediction for {company_name}')
    ax.legend()

    plt.xticks(rotation=45)
    plt.tight_layout()  # Adjust layout to make sure everything fits

    # Clear the previous canvas (if any) and display the new plot
    for widget in canvas_frame.winfo_children():
        widget.destroy()

    # Embed the plot into Tkinter canvas
    canvas = FigureCanvasTkAgg(fig, master=canvas_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()

    # Display the metrics below the plot
    metrics_label = tk.Label(canvas_frame, text=f"Predicted Price: ₹{predicted_price:.2f}\n"
                                                f"Amount Predicted: ₹{amount_predicted:.2f}\n"
                                                f"MSE: {mse:.2f}\n"
                                                f"MAE: {mae:.2f}", font=("Arial", 12))
    metrics_label.pack()

#  company selection and plotting
def on_company_select():
    company_name = company_name_var.get()
    try:
        amount = int(amount_entry.get())
    except ValueError:
        messagebox.showerror("Invalid input", "Please enter a valid amount")
        return

    if company_name not in final_dataset['Company'].values:
        messagebox.showerror("Error", f"Company '{company_name}' not found")
        return

    company_data = final_dataset[final_dataset['Company'] == company_name]

    predicted_price, amount_predicted, mse, mae = predict_stock_price(company_name, amount)

    if predicted_price is not None:
        plot_stock_chart(company_name, amount_predicted, predicted_price, mse, mae, company_data, canvas_frame)

def update_company_list():
    letter = company_letter_entry.get().upper()
    if letter:  # Check if the letter is not empty
        companies = final_dataset[final_dataset['Company'].str.startswith(letter, na=False)]['Company'].unique()
        companies.sort()  # Sort the companies alphabetically
        company_name_var.set('')  # Reset the company selection
        company_dropdown['menu'].delete(0, 'end')  # Clear the dropdown options
        for company in companies:
            company_dropdown['menu'].add_command(label=company, command=tk._setit(company_name_var, company))

# Create the main window using Tkinter
root = tk.Tk()
root.title("Stock Price Prediction")

# Create and place the company letter label and entry field
company_letter_label = tk.Label(root, text="Enter First Letter of Company:")
company_letter_label.grid(row=0, column=0)
company_letter_entry = tk.Entry(root)
company_letter_entry.grid(row=0, column=1)

# Create and place the company dropdown label and entry field
company_name_label = tk.Label(root, text="Select Company:")
company_name_label.grid(row=1, column=0)
company_name_var = tk.StringVar()
company_dropdown = tk.OptionMenu(root, company_name_var, [])
company_dropdown.grid(row=1, column=1)

# Add a button to update the company list based on the letter entered
update_button = tk.Button(root, text="Update Company List", command=update_company_list)
update_button.grid(row=2, columnspan=2)

# Create and place the amount label and entry field
amount_label = tk.Label(root, text="Enter Amount (Days):")
amount_label.grid(row=3, column=0)
amount_entry = tk.Entry(root)
amount_entry.grid(row=3, column=1)

# Create and place the submit button
submit_button = tk.Button(root, text="Predict Stock Price", command=on_company_select)
submit_button.grid(row=4, columnspan=2)

# Frame to display the plot inside the Tkinter window
canvas_frame = tk.Frame(root)
canvas_frame.grid(row=5, columnspan=2)

# Start the Tkinter event loop
root.mainloop()


C:\Users\Pc\AppData\Local\Temp\ipykernel_15092\3929163784.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_data['Date'] = pd.to_datetime(company_data['Date'])
C:\Users\Pc\AppData\Local\Temp\ipykernel_15092\3929163784.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_data['Date'] = company_data['Date'].map(pd.Timestamp.toordinal)
G:\anaconda\anaconda notebook\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with fe